In [1]:
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *

import sys

import pandas as pd


df = pd.read_csv('vocabulary.csv')
col_name = df.columns.tolist()

if 'index' not in col_name:
    a=[]
    for i in range(len(df)):
        a.append(i)
    df.insert(loc=0,column = 'index',value=a)
if 'proficiency' not in col_name:
    df['proficiency']=None
    
df = df[df['proficiency'] != 1]

def words_index(number):
    word_index = []
    for index in df.sample(n=number).index:
        word_index.append(index)
    return word_index

#Build a function that performs as a quiz and the form of the quiz is mutiple choice.
def quiz(number,df):
    import random
    result = {}
    # Quiz the words whose proficiency is not 1
    unknown_df = df[(df['proficiency'].notnull()) & (df['proficiency'] != 1)]
    print(unknown_df)
    #Quiz words whose proficiency is not 1 first
    if len(unknown_df) >= number:
        quiz_df = unknown_df
    elif len(unknown_df) >= 1:
        quiz_df = pd.concat([unknown_df, df[df['proficiency'].isnull()]])[: number + 1]
    # Else: Quiz the whole word list
    else:
        quiz_df = df            
    print(quiz_df)
    
    for i in range(number):
        r = []
        choices = list(df.sample(n=3, replace = True)['definitions'])  #choose 3 definition  randomly from the whole word list.
        solution = quiz_df['definitions'][i]
        index = quiz_df['index'][i]
        # Produce choices in a random order.
        flag = random.randint(0,3)
        choices.insert(flag, solution)
        r.append(index)
        r.append(choices)
        r.append(flag)
        result[quiz_df['words'][i]] = tuple(r)

    return result

class QuizWidget(QWidget):
    #this class creates group of radio buttons from given list of labels
    #constructor
    def __init__(self, quiz_dict, *args, **kwargs):
        super(QWidget, self).__init__(*args, **kwargs)

        self.result_list=[]
        self.quiz_iter = iter(quiz_dict.items())
        self.i = next(self.quiz_iter)
        self.word,self.choices,self.index, self.answer = self.i[0],self.i[1][1],self.i[1][0], self.i[1][2]
        self.instruction = QLabel('Please select the correct definition for: '+self.word)
        # self.instruction.setAlignment(Qt.AlignCenter)
        self.radio_button_group = QButtonGroup()

        #create radio buttons
        self.radio_button_list=[]
        for each in self.choices:
            self.radio_button_list.append(QRadioButton(each))

        #create layout for radio buttons and add them
        self.radio_button_layout = QVBoxLayout()

        #add buttons to the layout and button group
        counter = 0
        for each in self.radio_button_list:
            self.radio_button_layout.addWidget(each)
            self.radio_button_group.addButton(each)
            self.radio_button_group.setId(each, counter)
            counter += 1

        self.radio_button_group.buttonClicked.connect(self.button_clicked)

        self.mainlayout = QVBoxLayout()
        self.mainlayout.addWidget(self.instruction)
        self.mainlayout.addLayout(self.radio_button_layout)
        self.mainlayout.setAlignment(Qt.AlignCenter)
        self.setLayout(self.mainlayout)

    #method to find out the selected button
    def button_clicked(self):
        s = self.radio_button_group.checkedId()
        correct_ans = self.choices[self.answer]
        user_ans = self.choices[s]
        if s == self.answer:
            self.result_list.append((self.word,correct_ans,'Correct'))
            f = 1
        else:
            self.result_list.append((self.word,correct_ans,user_ans))
            f = -1
        if df['proficiency'][self.index]:
            df['proficiency'][self.index]+=f
        else:
            df['proficiency'][self.index]=f  

        self.next_quiz()


    def next_quiz(self):
        try:
            self.i = next(self.quiz_iter)
        except:
            print('done')
            for i in range(4):
                self.radio_button_list[i].hide()
            self.instruction.setText('Great work! Here are your results:')
            print(self.result_list)
            self.mainlayout.addWidget(ResultWidget(self.result_list))
            return
        
        self.word,self.choices,self.index, self.answer = self.i[0],self.i[1][1],self.i[1][0], self.i[1][2]
        self.instruction.setText('Please select the correct definition for: '+self.word)

        for i in range(4):
            self.radio_button_list[i].setText(self.choices[i])


class MemorizeWidget(QWidget):
    def __init__(self, quiz_num=10, *args, **kwargs):
        super(QWidget, self).__init__(*args, **kwargs)
        
        #layout for memorizing mode
        buttonlayout = QHBoxLayout()
        button_namelist = ["I don't know","Looks familiar","I know this"]
        self.button_list = []
        buttonlayout.setContentsMargins(25,25,25,25) 
        i = -1
        for b in button_namelist:
            button = QPushButton(b)
            self.button_list.append(button)
            button.pressed.connect(lambda i = i: self.btn_pushed(i))
            buttonlayout.addWidget(button)
            i+=1
        
        #layout for display screen (flashcard)
        self.quiz_num = quiz_num
        self.index_list = words_index(self.quiz_num)
        self.displaylayout = QVBoxLayout()
        self.displaylayout.setContentsMargins(50,50,75,50)
        self.selected_index = 0
        self.card = QPushButton(df['words'][self.index_list[self.selected_index]])
        self.card.setMinimumWidth(800)
        self.card.setMinimumHeight(300)
        self.card.clicked.connect(lambda x: self.card_clicked(self.card))
        self.displaylayout.addWidget(self.card)
        self.displaylayout.addLayout(buttonlayout)
        
        self.setLayout(self.displaylayout)
            
    def btn_pushed(self,i):
        if self.selected_index == self.quiz_num:
            print(df.iloc[self.index_list])
            df.to_csv('vocabulary.csv')
        else:
            if df['proficiency'][self.index_list[self.selected_index]]==None:
                df['proficiency'][self.index_list[self.selected_index]]= i
            else:
                df['proficiency'][self.index_list[self.selected_index]]+= i
            self.selected_index+=1
            self.card_clicked(self.card)

    def card_clicked(self,c):
        if self.selected_index >= self.quiz_num:
            c.setText('You have finished all the words for this session!')
            df.to_csv('vocabulary.csv')
            for i in self.button_list:
                i.hide()

        elif c.text() == df['words'][self.index_list[self.selected_index]]:
            c.setText('Definition:\n'+df['definitions'][self.index_list[self.selected_index]])
        else:
            c.setText(df['words'][self.index_list[self.selected_index]])

class CustomDialog(QInputDialog):
    def __init__(self, text, *args, **kwargs):
        super(CustomDialog, self).__init__(*args, **kwargs)

        self.setComboBoxItems([str(i) for i in range(10,60,10)])
        self.setLabelText(text)

class InitialWidget(QWidget):
    def __init__(self, callbackFunc, *args, **kwargs):
        super(QWidget, self).__init__(*args, **kwargs)

        self.mySrc = Communicate()
        self.mySrc.myGUI_signal.connect(callbackFunc) 

        #button layout
        memorize_btn = QPushButton('Memorize')
        memorize_btn.clicked.connect(lambda x: self.btn_clicked('memorize'))
        quiz_btn = QPushButton('Quiz')
        quiz_btn.clicked.connect(lambda x: self.btn_clicked('quiz'))
        button_layout = QHBoxLayout()
        button_layout.addWidget(memorize_btn)
        button_layout.addWidget(quiz_btn)


        initial_layout = QVBoxLayout()
        # initial_layout.setAlignment(Qt.AlignTop)
        initial_layout.setSpacing(100)
        initial_layout.setContentsMargins(150,150,150,150)
        welcome = QLabel("Welcome to WordPass, your awesome vocabulary builder")
        welcome.setAlignment(Qt.AlignCenter)
        font = QFont("Times", 15)
        welcome.setFont(font)
        description = QLabel(" Please select your goal today.")
        description.setAlignment(Qt.AlignCenter)
        initial_layout.addWidget(welcome)
        initial_layout.addWidget(description)
        initial_layout.addLayout(button_layout)

        self.setLayout(initial_layout)

    def btn_clicked(self,s):
        dlg = CustomDialog('What would you like to '+s+' today?')
        if dlg.exec_():
            i = int(dlg.textValue())
            if s=='memorize':
                self.mySrc.myGUI_signal.emit('0'+str(i))
                print(s)
            elif s=='quiz':
                self.mySrc.myGUI_signal.emit('1'+str(i))

class Communicate(QObject):
    myGUI_signal = pyqtSignal(str)

class ResultWidget(QWidget):
    def __init__(self, result_list, *args, **kwargs):
        super(QWidget, self).__init__(*args, **kwargs)

        #create table for result view
        self.table = QTableWidget()
        self.table.setColumnCount(3)
        self.table.setHorizontalHeaderLabels(["Word","Correct Answer","Your Answer"])

        self.table.setRowCount(len(result_list))
        for i in range(len(result_list)):
            for j in range(len(result_list[i])):
                s = QTableWidgetItem(result_list[i][j])
                self.table.setItem(i,j,s)

        self.table.resizeColumnsToContents()

        # set result widget layout
        self.layout = QVBoxLayout()
        self.layout.setSpacing(50)
        self.layout.setAlignment(Qt.AlignHCenter)
        self.layout.setContentsMargins(50,50,50,50)
        self.layout.addWidget(self.table)
        self.setLayout(self.layout)

#our class: MainWindow; qt parent class: QMainWindow
class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        #must always call the super init function or it won't work
        super(MainWindow, self).__init__(*args,**kwargs)

        #set window title, function of QMainWindow Class
        self.setWindowTitle("WordPass: My Awesome Vocabulary Builder")

        self.setCentralWidget(InitialWidget(callbackFunc= self.my_signal))

        home_btn= QAction( "&Home", self)
        home_btn.setStatusTip("Exit current session and go back to home page.")           
        home_btn.triggered.connect(self.home_clicked)
        # gen_new.setCheckable(True)
        home_btn.setShortcut(QKeySequence("Ctrl+h"))

        #set status bar
        self.setStatusBar(QStatusBar(self))

        #add menu 
        menu = self.menuBar()                               
        menu.addAction(home_btn)  
        menu.setNativeMenuBar(False)

    def home_clicked(self):
        self.setCentralWidget(InitialWidget(callbackFunc= self.my_signal))

    def my_signal(self,s):
        f,i = int(s[0]),int(s[1:])
        if f==0:
            self.setCentralWidget(MemorizeWidget(quiz_num = int(s[1:])))
        elif f==1:
            x= quiz(i,df=pd.read_csv('vocabulary.csv'))
            self.setCentralWidget(QuizWidget(quiz_dict = x))

#sys.argv allows pass command line arguments to application
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)

#create window instance (after create QApplication instantce but before enter event loop)
window = MainWindow() 
window.show()

#enter the event loop

app.exec_()

memorize


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


      Unnamed: 0  index        words  \
209          209    209  countervail   
404          404    404       fecund   
611          611    611    irascible   
788          788    788    platitude   
1008        1008   1008   stigmatize   
1040        1040   1040       supine   
1143        1143   1143   vituperate   

                                         definitions  proficiency  
209                        counterbalance                    -1.0  
404                               fertile                    -1.0  
611                irritable easily angered                   0.0  
788   a trite or banal statement unoriginality               -1.0  
1008                describe smb scornfully                  -1.0  
1040       lying on the back slow to act passive             -1.0  
1143                    curse abuse in words                 -1.0  
      Unnamed: 0  index        words  \
209          209    209  countervail   
404          404    404       fecund   
611          61

KeyError: 4

0